## Étude

In [ ]:
import numpy as np
import random
import cv2

#### NPS (Non-parametric sampling)

Algorithme:

In [6]:
import cv2
import numpy as np
import random

def growImage(sample_image, image, window_size):
    max_error = 0.3
    while not myCountNonZero(image) == image.shape[0] * image.shape[1]:
        pixel_list = getUnfilledNeighbours(image)
        if not pixel_list:
            break
        progress = 0
        for pixel in pixel_list:
            cv2.imshow("progress", image)
            cv2.waitKey(1)
            print(f"current pixel : {pixel[0]}x, {pixel[1]} y")
            window = np.array(getNeighbourWindow(image, pixel, window_size))
            matches = findMatches(sample_image, window, window_size, pixel)
            if matches:
                best_match = matches[random.randint(0, len(matches) - 1)]
                for ch in range(3):
                    image[pixel[0], pixel[1], ch] = sample_image[best_match[0], best_match[1], ch]
            cv2.destroyAllWindows()


def findMatches(sample_image, window, window_size, pixel):
    error_threshold = 0.3

    valid_mask = []
    columns = []
    for i in range(len(window)):
        columns = []
        for j in range(len(window)):
            if(window[i, j, :].any()):
                columns.append(1)
            else:
                columns.append(0)
        valid_mask.append(columns)

    gaussian_mask = get_gaussian2D(window.shape[0], window_size / 6.4)
    weight = np.sum(gaussian_mask * valid_mask)

    

    sum_sqred_diffs = np.zeros((sample_image.shape[0], sample_image.shape[1]))
    sum_sqred_diff_arrays = []

    for ch in range(3):
        sum_sqred_diffs = np.zeros((sample_image.shape[0], sample_image.shape[1]))
        iterations = 0
        for i in range(sample_image.shape[0]):
            for j in range(sample_image.shape[1]):
                for ii in range(window.shape[0]):
                    for jj in range(window.shape[1]):
                        if(valid_mask[ii][jj] == 0 or i - ii < 0 or j - jj < 0):
                            continue
                        try:
                            distance = pow((window[ii, jj, ch] - sample_image[i - ii, j - jj, ch]), 2)
                        except:
                            #ca a l'aire stupide mais a mon avis ca marche
                            distance = pow((window[ii, jj, ch] - sample_image[i + ii, j + jj, ch]), 2)

                        sum_sqred_diffs[i, j] += distance * gaussian_mask[ii][jj]
                sum_sqred_diffs[i, j] /= weight
        sum_sqred_diff_arrays.append(sum_sqred_diffs)
    
    resultList = []

    sum = sum_sqred_diff_arrays[0] + sum_sqred_diff_arrays[1] + sum_sqred_diff_arrays[2]
    ta = sum[window_size // 2 + 1 : - window_size // 2 - 1]
    ta = [row[window_size // 2 + 1 : - window_size // 2 - 1] for row in ta]

    min = np.min(ta)

    for i in range(window_size // 2, sample_image.shape[0] - window_size // 2 + 1):
        for j in range(window_size // 2, sample_image.shape[1] - window_size // 2 + 1):
            if(sum[i, j] != 0):
                if(sum[i, j] <= min * (1 + error_threshold)):
                    resultList.append((i, j))
                    print((i, j))

    return (resultList)

def get_gaussian2D(window_size, sigma):
    x, y = np.meshgrid(np.arange(window_size), np.arange(window_size) + 1)
    x = x - window_size//2
    y = y - window_size//2
    return 1/(2*np.pi*sigma**2) * np.exp(-(x**2 + y**2)/(2*sigma**2))

def getNeighbourWindow(image, pixel, window_size):
    window = []
    for i in range(max(0, pixel[0] - window_size // 2), min(pixel[0] + window_size // 2, image.shape[0]) + 1):
        rows = []
        for j in range(max(0, pixel[1] - window_size // 2), min(pixel[1] + window_size // 2, image.shape[1]) + 1):
            colors = []
            for k in range(3):
                colors.append(image[i, j, k])
            rows.append(colors)
        window.append(rows)
    return window

# recherche des pixels qui sont voisins a l'extremité de l'image
def getUnfilledNeighbours(image):
    result = []
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if((image[i, j, 0] == image[i, j, 1] == image[i, j, 2] == 0)):
                try:
                    if(image[i-1 : i+2, j-1 : j+2, :].any()):
                        result.append((i, j))
                except IndexError:
                    print("pixel n'existe pas")
    return result

def myCountNonZero(image):
    result = 0
    for j in range(image.shape[0]):
        for k in range(image.shape[1]):
            if(not image[j, k, :].any()):
                continue
            else:
                result += 1
    return result // 3

window_size = 8
sample_image = cv2.imread(".\\128x128\\brick\\Brick_06-128x128.png") / 255
image = np.zeros((sample_image.shape[0], sample_image.shape[1], 3))

image[sample_image.shape[0] // 2 - 6 : sample_image.shape[0] // 2 + 7, sample_image.shape[1] // 2 - 6 : sample_image.shape[1] // 2 + 7, :] = sample_image[sample_image.shape[0] // 2 - 6 : sample_image.shape[0] // 2 + 7, sample_image.shape[1] // 2 - 6 : sample_image.shape[1] // 2 + 7, :]

image = growImage(sample_image, image, window_size)

cv2.imwrite("sample.png", np.ciel(image * 255))
cv2.waitKey(0)
cv2.destroyAllWindows()

current pixel : 57x, 57 y
(17, 101)
(64, 66)
current pixel : 57x, 58 y
(66, 29)
(66, 112)
current pixel : 57x, 59 y
(66, 28)
(80, 108)
(83, 9)
current pixel : 57x, 60 y
(94, 120)
current pixel : 57x, 61 y
(61, 111)


IndexError: list index out of range

### Patch-based TS

In [ ]:
! ..\patch-based\texture_synthesis.py --image_path ../patch-based/data/input9.png --block_size 30 --num_block 14 --mode Best